In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch

In [ ]:

RESIZE = False

# create stereoscopic sbs pair from a single image
## read image
img = cv2.imread('input/input.jpg')
h, w = img.shape[:2]

# Resize image so that the largest dimension is MAX_RESOLUTION px
if RESIZE:
    MAX_RESOLUTION = 384
    max_dim = max(h, w)
    if max_dim > MAX_RESOLUTION:
        scale = MAX_RESOLUTION / max_dim
        img = cv2.resize(img, (int(w * scale), int(h * scale)), interpolation=cv2.INTER_AREA)
        h, w = img.shape[:2]

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title(f'Original Image ({w}x{h})')
plt.show()

In [ ]:
# Load MiDaS model
# midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
# midas = torch.hub.load('intel-isl/MiDaS', 'DPT_Hybrid')  # higher accuracy, but slower
midas = torch.hub.load('intel-isl/MiDaS', 'DPT_Large')  # highest accuracy, but slowest
midas.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
midas.to(device)

# Load transforms
transform = torch.hub.load('intel-isl/MiDaS', 'transforms').small_transform
# transform = torch.hub.load('intel-isl/MiDaS', 'transforms').dpt_transform # Higher resolution and depth map but bad edges for some reason

# Read image
# img = cv2.imread('input.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
input_batch = transform(img_rgb).to(device)

# Predict depth
with torch.no_grad():
    prediction = midas(input_batch)
    depth_map = prediction.squeeze().cpu().numpy()

# Normalize and display
depth_map = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
depth_map = depth_map.astype(np.uint8)
depth_map_size=depth_map.shape
# resize to original image size
depth_map = cv2.resize(depth_map, (w, h))
# cv2.imshow('Depth Map', depth_map)
cv2.imwrite('output/depth_map.jpg', depth_map)

# show depth map using matplotlib
plt.imshow(depth_map, cmap='gray')
plt.axis('off')
plt.title(f'Depth Map {depth_map_size}')
plt.show()

# produce img + depth map average
combined = cv2.addWeighted(img, 0.5, cv2.cvtColor(depth_map, cv2.COLOR_GRAY2BGR), 0.5, 0)
cv2.imwrite('output/combined.jpg', combined)
plt.imshow(cv2.cvtColor(combined, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Image + Depth Map Average')
plt.show()

In [ ]:
# Create stereoscopic SBS pair using depth map for pixel-wise shift
import numpy as np
import cv2

# Parameters
# max_shift = 30  # maximum pixel shift for closest objects (tune as needed)
max_shift_perc=2
max_shift = int(w * max_shift_perc / 100)  # e.g., 10% of smaller dimension
print(f'Max shift: {max_shift} pixels')

# Ensure depth_map and img are available from previous cells
h, w = depth_map.shape
left_img = np.zeros_like(img)
right_img = np.zeros_like(img)

# Normalize depth map to [0, 1]
depth_norm = cv2.normalize(depth_map.astype(np.float32), None, 0, 1, cv2.NORM_MINMAX)

# Smooth edges slightly
# depth_norm = cv2.bilateralFilter(depth_norm, d=9, sigmaColor=0.1, sigmaSpace=5)

for y in range(h):
    for x in range(w):
        shift = int(depth_norm[y, x] * max_shift)
        # Left image: shift pixel to the left
        lx = x - shift
        if 0 <= lx < w:
            left_img[y, lx] = img[y, x]
        # Right image: shift pixel to the right
        rx = x + shift
        if 0 <= rx < w:
            right_img[y, rx] = img[y, x]

#### FILL ####
# V0: FROM ORIGINAL: not working
# # Fill empty pixels (optional: inpaint or blur)
# left_img[left_img.sum(axis=2) == 0] = img[left_img.sum(axis=2) == 0]
# right_img[right_img.sum(axis=2) == 0] = img[right_img.sum(axis=2) == 0]

# V1: Simple propagation
# Fill blanks by propagating from left (for left_img) and right (for right_img)

# --- Left image: propagate values from left to right ---
for y in range(h):
    last_pixel = None
    for x in range(w):
        if np.all(left_img[y, x] == 0):  # blank
            if last_pixel is not None:
                left_img[y, x] = last_pixel
        else:
            last_pixel = left_img[y, x]

# --- Right image: propagate values from right to left ---
for y in range(h):
    last_pixel = None
    for x in range(w-1, -1, -1):  # sweep right→left
        if np.all(right_img[y, x] == 0):  # blank
            if last_pixel is not None:
                right_img[y, x] = last_pixel
        else:
            last_pixel = right_img[y, x]

# Concatenate and display
# sbs_depth = cv2.hconcat([left_img, right_img])
# create a 5% of width black border between images
sbs_depth = cv2.hconcat([left_img, np.zeros((h, int(w * 0.05), 3), dtype=np.uint8), right_img])
# cv2.imshow('Stereoscopic SBS from Depth', sbs_depth)
cv2.imwrite('output/stereoscopic_sbs_from_depth_pixelwise.jpg', sbs_depth)

plt.imshow(cv2.cvtColor(sbs_depth, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Stereoscopic SBS from Depth (Pixel-wise Shift)')
plt.show()

avg_img = cv2.addWeighted(left_img, 0.5, right_img, 0.5, 0)
# cv2.imshow('Average of L/R', avg_img)
cv2.imwrite('output/stereo_average.jpg', avg_img)

plt.imshow(cv2.cvtColor(avg_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Average of Left and Right')
plt.show()

# Absolute color difference between left and right
diff_img = cv2.absdiff(left_img, right_img)

# cv2.imshow('Difference (L vs R)', diff_img)
cv2.imwrite('output/stereo_difference.jpg', diff_img)

plt.imshow(cv2.cvtColor(diff_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title('Difference Between Left and Right')
plt.show()

In [ ]:
import imageio
import cv2

frames = [
    cv2.cvtColor(left_img, cv2.COLOR_BGR2RGB),
    cv2.cvtColor(right_img, cv2.COLOR_BGR2RGB)
]

# loop=0 → infinite loop
imageio.mimsave(
    'output/stereo_sbs.gif',
    frames,
    duration=0.5,
    loop=0
)

from IPython.display import Image
Image(filename='output/stereo_sbs.gif')
